<a href="https://colab.research.google.com/github/hycheungap/internship_keywords/blob/main/speech_keywords_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install pydub

Import Library

In [ ]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from pydub import AudioSegment

In [ ]:
m4a_audio = AudioSegment.from_file('/content/barackobama2004dncARXE.mp3', format='mp3')
#convert mp3 to wav to fit wav2vec file type
m4a_audio.export('/content/barackobama2004dncARXE.wav', format='wav')

In [ ]:
mp3_audio = AudioSegment.from_file('/content/barackobama2004dncARXE.wav', format='wav')
print(len(mp3_audio)/(1000*60))
# 15 Minutes audio breaks into 3 minutes 5 audio files (slicingis done by milliseconds)

counter_audio = 180
split_audio = [mp3_audio[:180*1000]]
for i in range(4):
    split_audio.append(mp3_audio[counter_audio*1000:(counter_audio+180)*1000])
    counter_audio += 180

count = 0
# # lets save it!
for count, audio_object in enumerate(split_audio):
    count += 1
    with open(f"{count}_audi_file.wav", 'wb') as out_f:
        audio_object.export(out_f, format='wav')

In [ ]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
# # load any audio file of your choice
collection_of_text = []
for i in range(4):

    speech, rate = librosa.load(f'{i+1}_audi_file.wav' ,sr=16000)

    input_values = tokenizer(speech, return_tensors='pt').input_values
    # Store logits (non-normalized predictions)
    with torch.no_grad():
        logits = model(input_values).logits

    # Store predicted id's
    predicted_ids = torch.argmax(logits, dim=-1)
    # decode the audio to generate text
    # Passing the prediction to the tokenzer decode to get the transcription
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    # transcriptions = tokenizer.decode(predicted_ids[0])
    print(transcription)
    collection_of_text.append(transcription)

print(collection_of_text)
final_complete_speech = ""

# convert batch of text into one complete sentence
for i in collection_of_text:
    final_complete_speech += i

print(final_complete_speech)

In [ ]:
# use this code to install Spacy, Scispacy and en_core_sci_scibert model
! pip install -U pip setuptools wheel
! pip install -U spacy
! pip install scispacy
! pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_scibert-0.4.0.tar.gz

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_sci_scibert')
doc = nlp(final_complete_speech.lower())
print(doc.ents)